<a href="https://colab.research.google.com/github/deep-diver/TFX-WandB/blob/main/notebooks/experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install tfx
!pip -q install keras_tuner
!pip -q install transformers
!pip -q install pillow
!pip -q install wandb
!pip -q install protobuf==3.20
!pip -q install --force-reinstall google_api_core
!pip -q install tensorflow

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
data_path = "gs://beans-lowres/tfrecords"
local_data_path = "data"

model_file = "modules/model.py"
model_fn = "modules.model.run_fn"
tuner_fn = "modules.model.tuner_fn"

proprocessing_file = "modules/preprocessing.py"
preprocessing_fn = "modules.preprocessing.preprocessing_fn"

schema_file = "schema.pbtxt"

In [2]:
!mkdir {local_data_path}
!gsutil cp {data_path}/*-00-*.tfrec {local_data_path}/

Copying gs://beans-lowres/tfrecords/train-00-32.tfrec...
Copying gs://beans-lowres/tfrecords/val-00-32.tfrec...
- [2 files][ 48.0 MiB/ 48.0 MiB]                                                
Operation completed over 2 objects/48.0 MiB.                                     


In [2]:
import tfx
import tensorflow

print(tfx.__version__, tensorflow.__version__)

1.12.0 2.11.0


In [3]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import ImportExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Tuner
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.proto import example_gen_pb2

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.dsl.components.common import resolver
from tfx.dsl.experimental.latest_blessed_model_resolver import LatestBlessedModelResolver

import tensorflow_model_analysis as tfma

In [4]:
import wandb 

WANDB_KEY = "0ec0fe10651e80585884c30f4c76f8175f25dff7"
wandb.Api(api_key=WANDB_KEY)

In [5]:
context = InteractiveContext()

In [6]:
input_config = example_gen_pb2.Input(
    splits=[
        example_gen_pb2.Input.Split(name="train", pattern="train-*.tfrec"),
        example_gen_pb2.Input.Split(name="eval", pattern="val-*.tfrec"),
    ]
)
example_gen = ImportExampleGen(
    input_base=local_data_path, 
    input_config=input_config
)

context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [7]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
context.show(statistics_gen.outputs['statistics'])

In [9]:
%%writefile {schema_file}

feature {
  name: "image"
  type: FLOAT

  presence {
    min_fraction: 1.0
  }

  float_domain {
    min: 0
    max: 255
  }

  shape {
    dim { size: 256 }
    dim { size: 256 }
    dim { size: 3 }
  }
}

feature {
  name: "image_shape"
  type: INT

  presence {
    min_fraction: 1.0
  }

  shape {
    dim { size: 3 }
  }
}

feature {
  name: "label"
  type: INT

  presence {
    min_fraction: 1.0
  }

  int_domain {
    min: 0
    max: 2
  }

  shape {
    dim { size: 1 }
  }  
}

Writing schema.pbtxt


In [10]:
schema_gen = tfx.components.ImportSchemaGen(
    schema_file=schema_file)

context.run(schema_gen)

ExecutionResult(
    component_id: ImportSchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=ImportSchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [11]:
example_validator = ExampleValidator(
  statistics=statistics_gen.outputs['statistics'],
  schema=schema_gen.outputs['schema']   
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
context.show(example_validator.outputs['anomalies'])

In [13]:
!mkdir modules

In [14]:
%%writefile {proprocessing_file}

import tensorflow as tf

IMAGE_KEY = "image"
LABEL_KEY = "label"

MODEL_INPUT_IMAGE_KEY = "pixel_values"
MODEL_INPUT_LABEL_KEY = "labels"

INPUT_IMG_SIZE = 224

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
      inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
      Map from string feature key to transformed feature operations.
    """
    # print(inputs)
    outputs = {}

    inputs[IMAGE_KEY] = tf.image.resize(
        inputs[IMAGE_KEY], [INPUT_IMG_SIZE, INPUT_IMG_SIZE]
    )

    inputs[IMAGE_KEY] = inputs[IMAGE_KEY] / 255.0
    inputs[IMAGE_KEY] = tf.transpose(inputs[IMAGE_KEY], [0, 3, 1, 2])

    outputs[MODEL_INPUT_IMAGE_KEY] = inputs[IMAGE_KEY]
    outputs[MODEL_INPUT_LABEL_KEY] = inputs[LABEL_KEY]

    return outputs

Writing modules/preprocessing.py


In [15]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    preprocessing_fn=preprocessing_fn)

In [16]:
context.run(transform)

Instructions for updating:
Use ref() instead.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [72]:
%%writefile {model_file}

from typing import List, Dict, Tuple

import absl
import tensorflow as tf
import keras_tuner
import tensorflow_transform as tft
from transformers import ViTFeatureExtractor, TFViTForImageClassification
from tfx.components.trainer.fn_args_utils import FnArgs
from tfx.v1.components import TunerFnResult

from tfx_bsl.tfxio import dataset_options
from tfx.components.trainer.fn_args_utils import DataAccessor

import wandb

feature_extractor = ViTFeatureExtractor()

_PROJECT_NAME = "TFX_WANDB_PILOT"

_TRAIN_LENGTH = 128
_EVAL_LENGTH = 128
_TRAIN_BATCH_SIZE = 8
_EVAL_BATCH_SIZE = 8

_EPOCHS = 30

_LABELS = ['angular_leaf_spot', 'bean_rust', 'healthy']

_CONCRETE_INPUT = "pixel_values"

_MODEL_INPUT_LABEL_KEY = "labels"

class MyHyperModel(keras_tuner.HyperModel):
  def build(self, hp):
    learning_rate = hp.get("learning_rate")

    id2label={str(i): c for i, c in enumerate(_LABELS)}
    label2id={c: str(i) for i, c in enumerate(_LABELS)}

    model = TFViTForImageClassification.from_pretrained(
      "google/vit-base-patch16-224-in21k",
      num_labels=len(_LABELS),
      label2id=label2id,
      id2label=id2label,
    )

    model.layers[0].trainable=False

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

    return model

class MyTuner(keras_tuner.RandomSearch):
  def __init__(self, *args, **kwargs):
    super().__init__(*args, **kwargs)

    self.trial_index = 0

  def run_trial(self, trial, *args, **kwargs):
    hp = trial.hyperparameters
    model = self.hypermodel.build(hp)

    log_name = f"hp-tuning-{self.trial_index}"
    self.trial_index = self.trial_index + 1

    wandb.init(
      project=_PROJECT_NAME, 
      config=hp.values,
    )

    result = self.hypermodel.fit(
      hp, 
      model, 
      *args, 
      callbacks=[wandb.keras.WandbMetricsLogger(log_freq='epoch')],
      **kwargs
    )

    wandb.finish()
    return result

def INFO(text: str):
    absl.logging.info(text)

def _normalize_img(
    img, mean=feature_extractor.image_mean, std=feature_extractor.image_std
):
    img = img / 255
    mean = tf.constant(mean)
    std = tf.constant(std)
    return (img - mean) / std


def _preprocess_serving(string_input):
    decoded_input = tf.io.decode_base64(string_input)
    decoded = tf.io.decode_jpeg(decoded_input, channels=3)
    resized = tf.image.resize(decoded, size=(224, 224))
    normalized = _normalize_img(resized)
    normalized = tf.transpose(
        normalized, (2, 0, 1)
    )  # Since HF models are channel-first.
    return normalized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def _preprocess_fn(string_input):
    decoded_images = tf.map_fn(
        _preprocess_serving, string_input, dtype=tf.float32, back_prop=False
    )
    return {_CONCRETE_INPUT: decoded_images}


def _model_exporter(model: tf.keras.Model):
    m_call = tf.function(model.call).get_concrete_function(
        tf.TensorSpec(
            shape=[None, 3, 224, 224], dtype=tf.float32, name=_CONCRETE_INPUT
        )
    )

    @tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
    def serving_fn(string_input):
        labels = tf.constant(list(model.config.id2label.values()), dtype=tf.string)
        images = _preprocess_fn(string_input)

        predictions = m_call(**images)
        indices = tf.argmax(predictions.logits, axis=1)
        pred_source = tf.gather(params=labels, indices=indices)
        probs = tf.nn.softmax(predictions.logits, axis=1)
        pred_confidence = tf.reduce_max(probs, axis=1)
        return {"label": pred_source, "confidence": pred_confidence}

    return serving_fn

###

def _transform_features_signature(
    model: tf.keras.Model, tf_transform_output: tft.TFTransformOutput
):
    """
    transform_features_signature simply returns a function that transforms
    any data of the type of tf.Example which is denoted as the type of sta
    ndard_artifacts.Examples in TFX. The purpose of this function is to ap
    ply Transform Graph obtained from Transform component to the data prod
    uced by ImportExampleGen. This function will be used in the Evaluator
    component, so the raw evaluation inputs from ImportExampleGen can be a
    pporiately transformed that the model could understand.
    """

    # basically, what Transform component emits is a SavedModel that knows
    # how to transform data. transform_features_layer() simply returns the
    # layer from the Transform.
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(
        input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")]
    )
    def serve_tf_examples_fn(serialized_tf_examples):
        """
        raw_feature_spec returns a set of feature maps(dict) for the input
        TFRecords based on the knowledge that Transform component has lear
        ned(learn doesn't mean training here). By using this information,
        the raw data from ImportExampleGen could be parsed with tf.io.parse
        _example utility function.
        Then, it is passed to the model.tft_layer, so the final output we
        get is the transformed data of the raw input.
        """
        feature_spec = tf_transform_output.raw_feature_spec()
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        return transformed_features

    return serve_tf_examples_fn


def _tf_examples_serving_signature(model, tf_transform_output):
    """
    tf_examples_serving_signature simply returns a function that performs
    data transformation(preprocessing) and model prediction in a sequential
    manner. How data transformation is done is idential to the process of
    transform_features_signature function.
    """

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function(
        input_signature=[tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")]
    )
    def serve_tf_examples_fn(
        serialized_tf_example: tf.Tensor,
    ) -> Dict[str, tf.Tensor]:
        raw_feature_spec = tf_transform_output.raw_feature_spec()
        raw_features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

        transformed_features = model.tft_layer(raw_features)
        logits = model(transformed_features).logits

        return {_MODEL_INPUT_LABEL_KEY: logits}

    return serve_tf_examples_fn

###

def _input_fn(
    file_pattern: List[str],
    data_accessor: DataAccessor,
    tf_transform_output: tft.TFTransformOutput,
    is_train: bool = False,
    batch_size: int = 32,
) -> tf.data.Dataset:
    INFO(f"Reading data from: {file_pattern}")

    dataset = data_accessor.tf_dataset_factory(
        file_pattern,
        dataset_options.TensorFlowDatasetOptions(
            batch_size=batch_size, label_key=_MODEL_INPUT_LABEL_KEY
        ),
        tf_transform_output.transformed_metadata.schema,
    )

    return dataset

def _get_hyperparameters() -> keras_tuner.HyperParameters:
    hp = keras_tuner.HyperParameters()
    hp.Choice("learning_rate", [1e-3, 1e-2], default=1e-3)
    return hp

def _build_model(hparams: keras_tuner.HyperParameters):
  learning_rate = hparams.get("learning_rate")

  wandb.init(
    project=_PROJECT_NAME, 
    config={
      "learning_rate": learning_rate
    },
    name=f"lr-{learning_rate}")

  id2label={str(i): c for i, c in enumerate(_LABELS)}
  label2id={c: str(i) for i, c in enumerate(_LABELS)}

  model = TFViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(_LABELS),
    label2id=label2id,
    id2label=id2label,
  )

  model.layers[0].trainable=False

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

  return model  

def run_fn(fn_args: FnArgs):
    wandb.login(key="")

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=True,
        batch_size=_TRAIN_BATCH_SIZE,
    )

    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=False,
        batch_size=_EVAL_BATCH_SIZE,
    )

    wandb.init(
      project=_PROJECT_NAME, 
      config=fn_args.hyperparameters.values,
      name="full-training"
    )

    hparams = keras_tuner.HyperParameters.from_config(fn_args.hyperparameters)
    INFO(f"HyperParameters for training: {hparams.get_config()}")
    model = MyHyperModel().build(hparams)

    model.fit(
        train_dataset,
        steps_per_epoch=_TRAIN_LENGTH // _TRAIN_BATCH_SIZE,
        validation_data=eval_dataset,
        validation_steps=_EVAL_LENGTH // _EVAL_BATCH_SIZE,
        epochs=_EPOCHS,
        callbacks=[wandb.keras.WandbMetricsLogger(log_freq='epoch')],
    )

    wandb.finish()

    model.save(
        fn_args.serving_model_dir, 
        save_format="tf", 
        signatures={
            "serving_default": _model_exporter(model),
            "transform_features": _transform_features_signature(
                model, tf_transform_output
            ),
            "from_examples": _tf_examples_serving_signature(model, tf_transform_output),
        },
    )

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
    wandb.login(key="")

    tuner = MyTuner(
        MyHyperModel(),
        max_trials=6,
        hyperparameters=_get_hyperparameters(),
        allow_new_entries=False,
        objective=keras_tuner.Objective("val_accuracy", "max"),
        directory=fn_args.working_dir,
        project_name=_PROJECT_NAME,
    )

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = _input_fn(
        fn_args.train_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=True,
        batch_size=_TRAIN_BATCH_SIZE,
    )

    eval_dataset = _input_fn(
        fn_args.eval_files,
        fn_args.data_accessor,
        tf_transform_output,
        is_train=False,
        batch_size=_EVAL_BATCH_SIZE,
    )

    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "x": train_dataset,
            "validation_data": eval_dataset,
            "epochs": 5,
            "steps_per_epoch": _TRAIN_LENGTH // _TRAIN_BATCH_SIZE,
            "validation_steps": _EVAL_LENGTH // _EVAL_BATCH_SIZE,
        },
    )


Overwriting modules/model.py


In [73]:
tuner = Tuner(
    tuner_fn=tuner_fn,
    examples=transform.outputs["transformed_examples"],
    schema=schema_gen.outputs["schema"],
    transform_graph=transform.outputs["transform_graph"],
)

In [ ]:
context.run(tuner, enable_cache=False)

In [ ]:
%wandb chansung18/TFX_WANDB_PILOT -h 800

In [ ]:
trainer = Trainer(
    run_fn=model_fn,
    transformed_examples=transform.outputs["transformed_examples"],
    transform_graph=transform.outputs["transform_graph"],
    schema=schema_gen.outputs["schema"],
    hyperparameters=tuner.outputs["best_hyperparameters"],
)

In [ ]:
context.run(trainer)

Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/bias:0', 'vit/pooler/dense/kernel:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


16/16 [==============================] - 12s 360ms/step - loss: 0.4430 - accuracy: 0.8594 - val_loss: 0.4070 - val_accuracy: 0.8438


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Instructions for updating:
Use fn_output_signature instead


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [ ]:
model_resolver = resolver.Resolver(
    strategy_class=LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing),
).with_id("latest_blessed_model_resolver")

In [ ]:
context.run(model_resolver)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [ ]:
eval_configs = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(
            signature_name="from_examples",
            preprocessing_function_names=["transform_features"],
            label_key="labels",
            prediction_key="labels",
        )
    ],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(
            metrics=[
                tfma.MetricConfig(
                    class_name="SparseCategoricalAccuracy",
                    threshold=tfma.MetricThreshold(
                        value_threshold=tfma.GenericValueThreshold(
                            lower_bound={"value": 0.55}
                        ),
                        # Change threshold will be ignored if there is no
                        # baseline model resolved from MLMD (first run).
                        change_threshold=tfma.GenericChangeThreshold(
                            direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute={"value": -1e-3},
                        ),
                    ),
                )
            ]
        )
    ],
)


evaluator = Evaluator(
    examples=example_gen.outputs["examples"],
    model=trainer.outputs["model"],
    baseline_model=model_resolver.outputs["model"],
    eval_config=eval_configs,
)

In [ ]:
context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [ ]:
!ls /tmp/tfx-interactive-2022-09-29T04_00_41.730943-z1asg51a/Evaluator/blessing/9

BLESSED
